In [1]:
from swarmauri.documents.concrete.Document import Document
from swarmauri.vector_stores.concrete.TfidfVectorStore import TfidfVectorStore

# Initialize the vector store
vector_store = TfidfVectorStore()

# Sample documents containing knowledge
documents = [
    Document(content="They use AWS for cloud services."),
    Document(content="They use PostgreSQL for their database."),
    Document(content="They code their backend in Python."),
    Document(content="They build their frontend with React."),
]

# Add documents to the vector store
vector_store.add_documents(documents)

# Verify the documents have been added
print(f"{len(vector_store.documents)} documents added to the vector store.")

# Retrieve and print all documents
all_docs = vector_store.get_all_documents()

print("All documents in the vector store:")
for doc in all_docs:
    print(doc.content)


4 documents added to the vector store.
All documents in the vector store:
They use AWS for cloud services.
They use PostgreSQL for their database.
They code their backend in Python.
They build their frontend with React.


In [2]:
from swarmauri.conversations.concrete.MaxSystemContextConversation import MaxSystemContextConversation
from swarmauri.messages.concrete.SystemMessage import SystemMessage
from swarmauri.messages.concrete.HumanMessage import HumanMessage

# Create a system message
system_context = SystemMessage(content="Your name is Jeff.")

# Initialize the conversation
conversation = MaxSystemContextConversation(system_context=system_context, max_size=4)

# Add a user message
user_message = HumanMessage(content="What is my name?")
conversation.add_message(user_message)

# Print the current conversation context
print("Current conversation history:")
for message in conversation.history:
    print(message.content)


Current conversation history:
Your name is Jeff.
What is my name?


In [4]:
import os
from swarmauri.llms.concrete.GroqModel import GroqModel as LLM
from swarmauri.conversations.concrete.Conversation import Conversation
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Retrieve the API key from environment variables
API_KEY = os.getenv("GROQ_API_KEY")

# Check if the API key is set
if not API_KEY:
    print("API key is not set. Please set the GROQ_API_KEY environment variable.")


In [5]:
# Function to get allowed models, filtering out failing ones
def get_allowed_models(llm):
    failing_llms = [
        "llama3-70b-8192",
        "llama-3.2-90b-text-preview",
        "mixtral-8x7b-32768",
        "lava-v1.5-7b-4096-preview",
        "llama-guard-3-8b",
    ]
    return [model for model in llm.allowed_models if model not in failing_llms]

# Initialize the GroqModel
llm = LLM(api_key=API_KEY)

# Print model information
print(f"Resource: {llm.resource}")
print(f"Type: {llm.type}")
print(f"Default Name: {llm.name}")

# Get allowed models
allowed_models = get_allowed_models(llm)
print("Allowed Models:", allowed_models)


Resource: LLM
Type: GroqModel
Default Name: gemma-7b-it
Allowed Models: ['gemma-7b-it', 'gemma2-9b-it', 'llama-3.1-70b-versatile', 'llama-3.1-8b-instant', 'llama-3.2-11b-text-preview', 'llama-3.2-1b-preview', 'llama-3.2-3b-preview', 'llama3-8b-8192', 'llama3-groq-70b-8192-tool-use-preview', 'llama3-groq-8b-8192-tool-use-preview', 'llava-v1.5-7b-4096-preview']


In [41]:
# Example Usage with No System Context
# Set the model name to the first available allowed model
llm.name = allowed_models[0]

# Create a conversation
conversation = Conversation()

# Add a human message
input_data = "Hello"
human_message = HumanMessage(content=input_data)
conversation.add_message(human_message)

# Predict response
llm.predict(conversation=conversation)
prediction = conversation.get_last().content
print(f"Prediction with no system context for {llm.name}: {prediction}")

# Example Usage with a System Context
# Example usage with a system context
system_context = "You are an assistant that provides answers to the user."
conversation = MaxSystemContextConversation(system_context=SystemMessage(content=system_context), max_size=1000)

# Create a human message with the content "Hi" and add it to the conversation
human_message = HumanMessage(content="Hi")
conversation.add_message(human_message)

# Predict response
llm.predict(conversation=conversation)
prediction = conversation.get_last().content
print(f"Prediction with system context for {llm.name}: {prediction}")


Prediction with no system context for gemma-7b-it: Hello! 👋 It's great to hear from you. How can I help you today? 😊
Prediction with system context for gemma-7b-it: Greetings! I am ready to assist you. How can I help you today? Please feel free to ask me any questions or request information on any topic. I will do my best to provide accurate and helpful responses.


In [42]:
from swarmauri.agents.concrete.RagAgent import RagAgent

# Create a new system context for the RAG Agent
rag_system_context = "You are an assistant that provides answers to the user. You utilize the details below:"
rag_conversation = MaxSystemContextConversation(
    system_context=SystemMessage(content=rag_system_context)
)

# Initialize the RAG Agent by combining LLM, conversation, and vector store
rag_agent = RagAgent(
    llm=llm,
    conversation=rag_conversation,
    system_context=rag_system_context,
    vector_store=vector_store,
)

# Handling Queries with the RAG Agent
# Test the agent with different queries
queries = [
    # "What cloud service do they use?",
    # "What database do they use?",
    "What is the SONY work experience of the Person in the Resume?"
]

for query in queries:
    response = rag_agent.exec(query)
    print(f"Query: {query}\nRAG Agent Response: {response}\n")


Query: What is the SONY work experience of the Person in the Resume?
RAG Agent Response: The provided text does not include any specific information regarding the person's current or previous work experience at Sony, so I am unable to extract the requested data from the given context.



In [10]:
from swarmauri.chunkers.concrete.SentenceChunker import SentenceChunker
chunker = SentenceChunker() 
print(f'resource:{chunker.resource}, type:{chunker.type}')

resource:Chunker, type:SentenceChunker


In [12]:
unchunked_text = 'A walk in the park is a nice start. After the walk, let us talk.'
chunks = ['A walk in the park is a nice start.', 'After the walk, let us talk.']
chunker.chunk_text(unchunked_text) == chunks

['A walk in the park is a nice start.', 'After the walk, let us talk.']

In [16]:
from pdfminer.high_level import extract_text
from pdfminer.layout import LAParams

def extract_text_from_pdf(pdf_path):
    # Configure layout analysis parameters
    laparams = LAParams(
        line_margin=0.5,      # Margin between lines
        word_margin=0.1,      # Margin between words
        char_margin=2.0,      # Margin between characters
        boxes_flow=0.5,       # Text flow direction
        detect_vertical=True  # Detect vertical text
    )
    
    # Extract text with custom layout parameters
    text = extract_text(
        pdf_path,
        laparams=laparams,
        codec='utf-8'
    )
    
    return text

# Example usage
pdf_path = "Resume_CV-Nilay Kumar.pdf"
formatted_text = extract_text_from_pdf(pdf_path)
print(formatted_text)

nnilayy.work@gmail.com | LinkedIn | github.com/nnilayy | Certifications | +91-6377778313

Nilay Kumar

SUMMARY
Passionate Software Engineer with expertise in Full-Stack Development and Machine Learning, with 1+ years of industrial experi-
ence at SONY. Skilled in building and deploying scalable web applications, integrating intuitive front-end designs with efficient and
secure back-end systems. Research experience spans ML projects under prestigious universities, with a focus on EDA, data prepro-
cessing, and developing practical ML solutions. Proficient in designing APIs, database management, and leveraging cloud platforms
for application deployment. Committed to continuous learning and adopting best practices in Full-Stack Development and Software
Engineering.

SKILLS

(cid:35)

Languages: Python, C, C++, C#, Java
ML, DL & Data Science: TensorFlow, PyTorch, Keras, Scikit-Learn, XGBoost, LightGBM, ONNX, IMB-Learn, OpenCV,

(cid:35)
Pandas, NumPy, Seaborn, Matplotlib, NLTK, Spacy, MATL

In [18]:
chunker.chunk_text(formatted_text)


In [25]:


# Verify the documents have been added
print(f"{len(vector_store.documents)} documents added to the vector store.")

# Optional: Print first few documents to verify content
print("\nSample documents in the vector store:")
for doc in list(vector_store.get_all_documents())[:]:  # Show first 3 docs
    print(f"- {doc.content}")

129 documents added to the vector store.

Sample documents in the vector store:
- They use AWS for cloud services.
- They use PostgreSQL for their database.
- They code their backend in Python.
- They build their frontend with React.
- nnilayy.work@gmail.com | LinkedIn | github.com/nnilayy | Certifications | +91-6377778313

Nilay Kumar

SUMMARY
Passionate Software Engineer with expertise in Full-Stack Development and Machine Learning, with 1+ years of industrial experi-
ence at SONY.
- Skilled in building and deploying scalable web applications, integrating intuitive front-end designs with efficient and
secure back-end systems.
- Research experience spans ML projects under prestigious universities, with a focus on EDA, data prepro-
cessing, and developing practical ML solutions.
- Proficient in designing APIs, database management, and leveraging cloud platforms
for application deployment.
- Committed to continuous learning and adopting best practices in Full-Stack Development and Softw